## 多层感知机实现

In [22]:
import torch
import time
import torchvision
import torchvision.transforms as transforms
from torch import nn
from torch.nn import init
import numpy as np
import sys
import torch.nn.functional as func

### 读取数据

In [30]:
def load_dataset(dataSet, batch_size=64):
    if sys.platform.startswith('win'):
        num_workers = 0  # 0表示不用额外的进程来加速读取数据
    else:
        num_workers = 4
        print('linux')
    dataset_iter = torch.utils.data.DataLoader(dataSet, pin_memory=False, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    return dataset_iter

### 评价准确率

In [31]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        X = X.cuda()
        y = y.cuda()
        acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
        n += y.shape[0]
    return acc_sum / n

### 定义模型

In [32]:
class multilp(nn.Module):
    def __init__(self, num_input, num_hidelayer1, num_output):
        super(multilp, self).__init__()
        self.hidelayer1 = nn.Linear(num_input, num_hidelayer1)
        self.outputlayer = nn.Linear(num_hidelayer1, num_output)
    def forward(self, x):
        x1 = func.relu(self.hidelayer1(x.view(x.shape[0], -1)))
        y = self.outputlayer(x1)
        return y

### 模型训练

In [33]:
def training_loop(net, loss, optimizer, num_epochs, train_dataset, test_dataset, batch_size):
    train_iter = load_dataset(train_dataset, batch_size)
    test_iter = load_dataset(test_dataset, batch_size)
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for X, y in train_iter:
            # print(X.device)
            X = X.cuda()
            y = y.cuda()
            y_hat = net(X)           # 前向传播  
            l = loss(y_hat, y).sum() # 计算损失
            optimizer.zero_grad()    # 梯度清零
            l.backward()             # 后向传播
            optimizer.step()         # 更新参数

            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
            n += y.shape[0]
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))

In [34]:
if __name__ == '__main__':
    savePath = './Fashion-MNIST/'
    mnist_train = torchvision.datasets.FashionMNIST(root=savePath, train=True, download=True, transform=transforms.ToTensor())
    mnist_test = torchvision.datasets.FashionMNIST(root=savePath, train=False, download=True, transform=transforms.ToTensor())
    
    # 初始化模型参数
    num_input = 784
    num_hidelayer1 = 256
    num_output = 10
    net1 = multilp(num_input, num_hidelayer1, num_output)
    cuda_gpu = torch.cuda.is_available()   #判断GPU是否存在可用
    if cuda_gpu:
        print("将模型迁移到cuda")
        torch.nn.DataParallel(net1, device_ids=[0]).cuda()
    for param in net1.parameters():
        init.normal_(param, mean=0, std=0.1)
    print(net1)  
    for param in net1.parameters():
        print(param)
    n_epochs=10
    optimizer = torch.optim.SGD(net1.parameters(), lr = 0.1)
    loss = torch.nn.CrossEntropyLoss(size_average=True)
    start = time.time()
    training_loop(net=net1, loss=loss, optimizer=optimizer, num_epochs=n_epochs, 
                train_dataset=mnist_train, test_dataset=mnist_test, batch_size=64)
    
    print(' %2f sec.' % (time.time() - start))

将模型迁移到cuda
multilp(
  (hidelayer1): Linear(in_features=784, out_features=256, bias=True)
  (outputlayer): Linear(in_features=256, out_features=10, bias=True)
)
Parameter containing:
tensor([[-0.1011,  0.0704, -0.0215,  ...,  0.0023, -0.0231, -0.1040],
        [ 0.2050,  0.0217, -0.1322,  ...,  0.0841, -0.0210,  0.0020],
        [ 0.0098, -0.1039, -0.0114,  ..., -0.0629,  0.0297, -0.0299],
        ...,
        [-0.0128,  0.1151,  0.1843,  ...,  0.0783,  0.1147,  0.0758],
        [ 0.0070,  0.1260,  0.0647,  ..., -0.2231,  0.0172,  0.0034],
        [ 0.1976, -0.0438,  0.0795,  ..., -0.1130, -0.1027, -0.0872]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 5.4432e-02, -8.0620e-02, -8.2507e-02, -2.1078e-02, -4.0511e-02,
        -5.3818e-03, -8.0918e-02, -2.7570e-02,  4.6572e-02, -1.4870e-01,
         8.7426e-02, -5.3194e-03,  5.6638e-03, -3.9338e-02, -1.5680e-02,
        -4.3042e-02,  1.2731e-01, -1.0435e-01,  6.9241e-02, -1.5639e-01,
         2.2956e-02,  1.10